# Agrupaciones de datos

Las agrupaciones son operaciones necesarias para analizar datos, ya que permiten extraer información en función de datos categóricos de nuestro dataframe.

Cargaremos los datos llamados _experiment.csv_ que podemos encontrar en el siguiente [enlace](https://raw.githubusercontent.com/bmalcover/MADM2019/master/data/experiment.csv)

In [2]:
import pandas as pd
import numpy as np

df= pd.read_csv("data/experiment.csv")
print(df)

   Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
0    Will     Smith    1.43    M  10/10/1920  0.19   laboral
1     Jon      Snow    1.98    M   10/1/1970  0.98   laboral
2    Laia   Ramirez    1.87    F  09/10/1987  0.76      cap6
3    Luzy      Raim    1.67    F  23/07/1979  0.56      cap6
4    Fein      Mang    1.78    M  12/03/1937  0.27      cap6
5  Victor     Colom    1.78    M  22/09/1957  0.97      cap8


En el siguiente ejemplo agrupamos los datos según el sexo de la persona mediante el método `groupby` que devuelve un `DataFrame` agrupado:

In [3]:
bySex = df.groupby('Sexo')
bySex

El atributo `groups` nos muestra los grupos hemos creado:

In [9]:
# Podemos saber los grupos realizados y que índices del dataframe tienen.
bySex.groups # nos proporciona un diccionario


{'F': [2, 3], 'M': [0, 1, 4, 5]}

In [10]:
bySex.describe()

Altura                                                     Cof          \
      count    mean       std   min     25%   50%   75%   max count    mean   
Sexo                                                                          
F       2.0  1.7700  0.141421  1.67  1.7200  1.77  1.82  1.87   2.0  0.6600   
M       4.0  1.7425  0.228674  1.43  1.6925  1.78  1.83  1.98   4.0  0.6025   

                                                
           std   min   25%   50%     75%   max  
Sexo                                            
F     0.141421  0.56  0.61  0.66  0.7100  0.76  
M     0.431383  0.19  0.25  0.62  0.9725  0.98

Esto nos permite realizar operaciones de filtrado con base a los grupos que hemos creado:

In [4]:
dfM = df.loc[bySex.groups['M'].values] #Recorda que "loc" accedeix per index de fila
dfM

Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
0    Will     Smith    1.43    M  10/10/1920  0.19   laboral
1     Jon      Snow    1.98    M   10/1/1970  0.98   laboral
4    Fein      Mang    1.78    M  12/03/1937  0.27      cap6
5  Victor     Colom    1.78    M  22/09/1957  0.97      cap8

In [7]:
df[df.Sexo=="M"] # es lo  mismo!

Nombre Apellidos  Altura Sexo  Nacimiento   Cof Categoria
0    Will     Smith    1.43    M  10/10/1920  0.19   laboral
1     Jon      Snow    1.98    M   10/1/1970  0.98   laboral
4    Fein      Mang    1.78    M  12/03/1937  0.27      cap6
5  Victor     Colom    1.78    M  22/09/1957  0.97      cap8

### Funciones de agregación en grupos.

El método 'aggregate' nos permite crear variables de agregación en la tabla obtenida con 'groupby'. Indicaremos la información que deseamos obtener de cada columna utilizando un diccionario. Especificamos la función que aplicaremos a los datos de cada grupo en cada columna para obtener un único valor.

- Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html

Para ejemplificar esta sección, agruparemos el _dataframe_ por `Categoria` laboral. En este caso para la columna `Altura` consultamos la suma de las alturas del grupo y `Cof` la media.

La función `aggregate` nos permite crear variables de agregación sobre la tabla obtenida con `groupby`. Indicaremos la información que queremos obtener de cada columna con un diccionario. Especificamos la función que vamos a aplicar a los datos de cada grupo en cada columna para obtener un único valor.

In [5]:
dfg = df.groupby(["Categoria"]).aggregate({
    "Altura":"sum",
    "Cof":"mean"})

dfg

Altura    Cof
Categoria               
cap6         5.32  0.530
cap8         1.78  0.970
laboral      3.41  0.585

In [12]:
dfg.index

Index(['cap6', 'cap8', 'laboral'], dtype='object', name='Categoria')

**Podemos aplicar un gran número de funciones de agregación:**

- [Funciones estadísticas](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html): mean, std, ...

- [Funciones matemáticas](https://docs.scipy.org/doc/numpy/reference/routines.math.html): sum, prod, ...

- Otras funciones: max, min, ...

- [Documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html)


In [9]:
type(dfg) #Alerta! Una agregación genera un dataframe y por lo tanto podemos seguir aplicando lo que ya sabemos

pandas.core.frame.DataFrame

In [10]:
dfg[dfg.Altura>3]

Altura    Cof
Categoria               
cap6         5.32  0.530
laboral      3.41  0.585

### Agrupaciones de múltiples columnas

También se pueden realizar agrupaciones de múltiples columnas. Se crean todas las combinaciones de las diversas columnas que existen en el DataFrame. Veamos un ejemplo:

In [7]:
gr = df.groupby(['Sexo',"Categoria"]).mean(numeric_only=True)
gr



Altura    Cof
Sexo Categoria               
F    cap6        1.770  0.660
M    cap6        1.780  0.270
     cap8        1.780  0.970
     laboral     1.705  0.585

Si queremos realizar un conteo de los elementos, debemos seleccionar

In [39]:
gr = df.groupby(['Sexo',"Categoria"])["Sexo"].count()
print(gr)

Sexo  Categoria
F     cap6         2
M     cap6         1
      cap8         1
      laboral      2
Name: Sexo, dtype: int64


### Multiindice 

A veces, un índice no es suficiente para expresar la meta-información que identifica una o varias columnas. Por ejemplo, una coordenada está compuesta por la latitud y la longitud.

Un 'multiíndice' es una jerarquía de índices.

Agrupar según diferentes criterios resulta en un multiíndice.

In [8]:
gr = df.groupby(['Sexo',"Categoria"]).count()
gr.index

MultiIndex([('F',    'cap6'),
            ('M',    'cap6'),
            ('M',    'cap8'),
            ('M', 'laboral')],
           names=['Sexo', 'Categoria'])

In [ ]:
gr.loc[("M",:)] # Como accedemos en rango en un multiindice? 

SyntaxError: invalid syntax (4256932143.py, line 1)

In [17]:
gr.loc["F"] # primer index

Nombre  Apellidos  Altura  Nacimiento  Cof
Categoria                                            
cap6            2          2       2           2    2

In [16]:
gr.loc["cap6"] # dependent index

KeyError: 'cap6'

In [18]:
# Necesitamos invocar un IndexSlice.
gr.loc[pd.IndexSlice[:, 'cap6'],:] 
# https://pandas.pydata.org/docs/reference/api/pandas.IndexSlice.html

Nombre  Apellidos  Altura  Nacimiento  Cof
Sexo Categoria                                            
F    cap6            2          2       2           2    2
M    cap6            1          1       1           1    1

En estas situaciones donde queremos acceder a esos datos, la forma más sencilla es simplemente eliminar el indice:

In [10]:
gr = df.groupby(['Sexo',"Categoria"]).count().reset_index()
gr

Sexo Categoria  Nombre  Apellidos  Altura  Nacimiento  Cof
0    F      cap6       2          2       2           2    2
1    M      cap6       1          1       1           1    1
2    M      cap8       1          1       1           1    1
3    M   laboral       2          2       2           2    2

In [21]:
gr.index = gr["Categoria"]
print(gr)

          Sexo Categoria  Nombre  Apellidos  Altura  Nacimiento  Cof
Categoria                                                           
cap6         F      cap6       2          2       2           2    2
cap6         M      cap6       1          1       1           1    1
cap8         M      cap8       1          1       1           1    1
laboral      M   laboral       2          2       2           2    2


### Agregaciones avanzadas

Sobre una agregación, podemos realizar operaciones más allá de las aritméticas. <br/>

Por ejemplo, si deseamos crear un histograma de la distribución de tipos de certificados por ciudad y código postal, necesitamos crear una lista para cada grupo.

In [11]:
df = pd.read_csv("data/data_groups.csv") 
print(df.head())
print(df.shape)


   Unnamed: 0        Dni                           Nom    CP      Ciutat Sexe  \
0           0  H61414629     María Dolores Arjona Jove  7800     Eivissa    M   
1           1  S3138381C                  Núria Quirós  7511     Ruberts    F   
2           2  J8698188C  Miguel José María Gil Vargas  7340       Alaro    M   
3           3  A48821615       Jordi Chaves Bustamante  7609  Bellavista    F   
4           4  U0247281I    Jana Rosa Collado Menéndez  7006       Palma    M   

  Tipus certificat cat  Punts  
0                    B     73  
1                    A     40  
2                    A     45  
3                    B     40  
4                    B     86  
(1000, 8)


In [22]:
dfc = df.groupby(["Ciutat","CP"])["Tipus certificat cat"].apply(list)
dfc

Ciutat      CP  
Alaro       7340    [A, C, B, A, A, B, A, D, A, B, A, B, B, A, B, ...
Ariany      7529    [B, A, B, A, A, B, A, B, A, B, A, A, C, A, B, ...
Bellavista  7609    [B, C, A, A, A, C, A, A, B, A, A, A, B, A, C, ...
Binissalem  7350    [A, B, B, C, C, A, A, A, C, A, C, A, B, B, D, ...
Eivissa     7800    [B, A, A, B, B, B, A, A, A, A, B, C, A, A, B, ...
La Savina   7870    [A, A, A, B, A, A, B, B, A, A, B, A, B, B, A, ...
Mao         7701    [B, C, B, C, B, C, A, B, B, B, A, C, C, A, C, ...
            7703    [B, A, B, C, A, A, B, A, C, B, A, B, C, A, B, ...
Palma       7006    [B, A, B, B, A, A, A, B, B, A, A, B, B, A, B, ...
            7009    [A, D, A, B, C, A, A, C, B, A, A, A, C, A, A, ...
            7013    [A, A, B, C, C, C, C, A, A, A, B, A, C, C, A, ...
Pedruscada  7590    [A, B, A, A, A, A, B, B, C, B, B, B, C, A, A, ...
Ruberts     7511    [A, A, A, B, C, A, B, A, A, B, B, A, C, B, B, ...
Name: Tipus certificat cat, dtype: object

In [23]:
tipusAlaro = dfc.loc[pd.IndexSlice["Alaro",7340]]

values, counts = np.unique(tipusAlaro, return_counts=True)
print(values) # Tipus 
print(counts) # quantitat
print("-"*40)
distribucioAlaro = dict(zip(values,counts)) ## Que fa el ZIP?!
print(distribucioAlaro)

['A' 'B' 'C' 'D']
[57 42 16  5]
----------------------------------------
{'A': 57, 'B': 42, 'C': 16, 'D': 5}


In [24]:
def hola(valor):
    print(valor*3)

In [25]:
hola(9)

27


In [24]:
## També podem invocar funcions especifiques! en lloc de una sum, mean, max, etc.
df2 = df.groupby(["Ciutat","CP"]).agg(
        {"Tipus certificat cat": [lambda x: list(x), np.size]}) # lambda !

print(df2)

                                              Tipus certificat cat     
                                                        <lambda_0> size
Ciutat     CP                                                          
Alaro      7340  [A, C, B, A, A, B, A, D, A, B, A, B, B, A, B, ...  120
Ariany     7529  [B, A, B, A, A, B, A, B, A, B, A, A, C, A, B, ...  113
Bellavista 7609  [B, C, A, A, A, C, A, A, B, A, A, A, B, A, C, ...   98
Binissalem 7350  [A, B, B, C, C, A, A, A, C, A, C, A, B, B, D, ...   93
Eivissa    7800  [B, A, A, B, B, B, A, A, A, A, B, C, A, A, B, ...   91
La Savina  7870  [A, A, A, B, A, A, B, B, A, A, B, A, B, B, A, ...   90
Mao        7701  [B, C, B, C, B, C, A, B, B, B, A, C, C, A, C, ...   63
           7703  [B, A, B, C, A, A, B, A, C, B, A, B, C, A, B, ...   45
Palma      7006  [B, A, B, B, A, A, A, B, B, A, A, B, B, A, B, ...   36
           7009  [A, D, A, B, C, A, A, C, B, A, A, A, C, A, A, ...   23
           7013  [A, A, B, C, C, C, C, A, A, A, B, A, C, C, A, .

In [41]:
# Qué és una funció lambda?

a = np.array([10,10,4,5,7,8,12,4507,30])
b = list(map(lambda i:i**2+2*i,a))

def f1(i):
    return i**2+2

b = list(map(f1,a))

print(b)

[102, 102, 18, 27, 51, 66, 146, 20313051, 902]


In [36]:
# Em aquest exemple apliquem una funció que té un criterí més específic:

def miBarem50p(serie):
  up50list = []
  for value in serie.values:
    if value>50:
      up50list.append(value)
  return len(up50list)


df3 = df.groupby(["Ciutat","CP"]).agg(
        {"Punts": [lambda x: miBarem50p(x), np.size]}) # lambda: què és X?

print(df3) #Quina interpretació dels resultats feu?

                     Punts     
                <lambda_0> size
Ciutat     CP                  
Alaro      7340         61  120
Ariany     7529         56  113
Bellavista 7609         43   98
Binissalem 7350         50   93
Eivissa    7800         44   91
La Savina  7870         41   90
Mao        7701         34   63
           7703         18   45
Palma      7006         19   36
           7009         10   23
           7013         19   37
Pedruscada 7590         65  106
Ruberts    7511         36   85


### Ejercicios

**1) Usando el fichero WHO.csv, ¿Cuál es el volumen total de CO2 emitido por cada continente?**

**2) ¿Cuál es el número de paises por continente?**

**3) Del conjunto "who.csv" selecciona 30 paises al azar y sobre ellos calcula la media de "Net primary school enrolment ratio female (%)" agrupados por: Continente**

Nota: la selección de 30 paises aleatoria ha de ser reproducible

**3b) Repite la anterior actividad pero ahora con todos los paises. ¿Sale la misma media?**

**4) Calcula la cantidad de ayuda recibida por cada municipio en función del númeto total de habitantes. (v2)**

In [1]:
# V2. Con necesidad de agrupar
import pandas as pd
import random

random.seed(0)

nombres = [f'Municipio{i}' for i in range(1, 11)] 

data_municipios = {
    'Nombre': nombres,
    'Código Postal': [random.randint(10000, 99999) for _ in range(10)],
    'Población': [random.randint(1000, 50000) for _ in range(10)]  # Añadimos un atributo aleatorio, en este caso "Población"
}

df_municipios = pd.DataFrame(data_municipios)


data_ayudas = {
    'Nombre': [random.choice(nombres) for _ in range(20)],
    'Ayuda Económica (en euros)': [random.randint(1000, 10000) for _ in range(20)],
    'Número de Beneficiarios': [random.randint(10, 100) for _ in range(20)]
}

df_ayudas = pd.DataFrame(data_ayudas)

print("Dataframe de Municipios:")
print(df_municipios)

print("\nDataframe de Ayudas:")
print(df_ayudas)

Dataframe de Municipios:
        Nombre  Código Postal  Población
0   Municipio1          60494      39232
1   Municipio2          65125      15315
2   Municipio3          15306      34075
3   Municipio4          43936      10127
4   Municipio5          77013      19470
5   Municipio6          73691      10158
6   Municipio7          63075       7214
7   Municipio8          49755      41525
8   Municipio9          72468      17417
9  Municipio10          56930      35902

Dataframe de Ayudas:
         Nombre  Ayuda Económica (en euros)  Número de Beneficiarios
0   Municipio10                        9989                       72
1    Municipio3                        1230                       23
2    Municipio5                        2528                       48
3    Municipio2                        7534                       80
4    Municipio2                        1018                       47
5    Municipio6                        9086                      100
6    Municipio8    

**5)  Agrupa los datos por el nombre de de la escuela.**
- ¿Qué escuela tiene más infantes?
- ¿Qué escuela tiene los infantes más altos?

In [ ]:
data = {
    'school': ['s001', 's002', 's003', 's001', 's002', 's004'],
    'class': ['V', 'V', 'VI', 'VI', 'V', 'VI'],
    'name': ['Alberto Franco', 'Gino Mcneill', 'Ryan Parkes', 'Eesha Hinton', 'Gino Mcneill', 'David Parkes'],
    'date_Of_Birth': ['15/05/2002', '17/05/2002', '16/02/1999', '25/09/1998', '11/05/2002', '15/09/1997'],
    'age': [12, 12, 13, 13, 14, 12],
    'height': [173, 192, 186, 167, 151, 159],
    'weight': [35, 32, 33, 30, 31, 32],
    'address': ['street1', 'street2', 'street3', 'street1', 'street2', 'street4']
}

df = pd.DataFrame(data, index=['S1', 'S2', 'S3', 'S4','S5','S6'])
df.head()

In [ ]:
#TODO dejar esto pendiente para isaac futuro
#hola
#HOLA


**6) Dado el siguiente listado de ventas:**
- ¿Qué comprador ha gastado más?
- ¿Qué vendedor ha hecho más ventas?

In [6]:
import pandas as pd
import numpy as np

# Crear datos aleatorios
np.random.seed(1)
n = 15
productos = ['Producto ' + str(i) for i in range(1, n+1)]
precios = np.random.randint(10, 100, n)
compradores = np.random.choice(['Juan', 'Pedro', 'Maria', 'Ana'], n)
vendedores = np.random.choice(['Carlos', 'Laura', 'Miguel', 'Elena'], n)

# Crear DataFrame
df_ventas = pd.DataFrame({
    'Producto': productos,
    'Precio': precios,
    'Comprador': compradores,
    'Vendedor': vendedores
})

print(df_ventas)

       Producto  Precio Comprador Vendedor
0    Producto 1      47      Juan    Laura
1    Producto 2      22     Maria    Laura
2    Producto 3      82     Pedro    Elena
3    Producto 4      19     Maria   Miguel
4    Producto 5      85      Juan   Carlos
5    Producto 6      15       Ana   Miguel
6    Producto 7      89      Juan    Laura
7    Producto 8      74     Maria    Laura
8    Producto 9      26      Juan    Laura
9   Producto 10      11     Pedro    Elena
10  Producto 11      86     Maria    Elena
11  Producto 12      81     Maria    Laura
12  Producto 13      16      Juan   Miguel
13  Producto 14      35       Ana    Laura
14  Producto 15      60       Ana    Laura


In [7]:
#TODO


**7) Fichero "data/data_groups_cursos.csv"; contiene cursos realizados por personas. Cada muestra corresponde a un curso.**
Se pide un listado del número de cursos realizado por cada persona y una valoración de sus puntos según el tipo de curso realizado:
- 'A': 3 puntos
- 'B': 2 puntos
- 'C': 1 punto
- 'D': 0.5 punto
- "E": 0.5 punto
- "F": 1 punto

In [8]:
# TODO

In [42]:
a = np.array([1,2,3,4,5,6,10])
print(a.max())
print(a.argmax())


10
6


# Categorización de datos

TODO
Valores ya categorizados?

https://pandas.pydata.org/docs/user_guide/categorical.html



#### Función `cut`
La función `cut` de Pandas es una herramienta útil para segmentar y discretizar datos en intervalos o categorías. Esta función es especialmente útil cuando desea convertir una variable numérica continua en una variable categórica al dividirla en intervalos o categorías específicas.

[Documentación](https://pandas.pydata.org/docs/reference/api/pandas.cut.html)

Crearemos un nuevo DataFrame para realizar pruebas:

In [3]:
import numpy as np
import pandas as pd
np.random.seed(0)
df_random = pd.DataFrame({"candidat":np.arange(1,11), "nota":np.random.randint(0,11,size=10)})
df_random

candidat  nota
0         1     5
1         2     0
2         3     3
3         4     3
4         5     7
5         6     9
6         7     3
7         8     5
8         9     2
9        10     4

A continuación, creamos una columna nueva que contiene la nota pero en escala categórica:

In [10]:
df_random["notaCategorica"] = pd.cut(df_random.nota, 3, labels=["dolent", "mig", "bona"])
df_random

candidat  nota notaCategorica
0         1     5            mig
1         2     0         dolent
2         3     3         dolent
3         4     3         dolent
4         5     7           bona
5         6     9           bona
6         7     3         dolent
7         8     5            mig
8         9     2         dolent
9        10     4            mig

In [12]:
df_random[df_random.notaCategorica==df_random.notaCategorica.max()]

candidat  nota notaCategorica
4         5     7           bona
5         6     9           bona

In [45]:
# mejora
bins = [-1, 4, 7, 8.5, 10]
labels = ['Suspens', 'Be', 'Notable', 'Excel·lent']
df_random["notaCategorica"] = pd.cut(df_random.nota, bins, labels=labels) # include_lowest=True
df_random

candidat  nota notaCategorica
0         1     5             Be
1         2     0        Suspens
2         3     3        Suspens
3         4     3        Suspens
4         5     7             Be
5         6     9     Excel·lent
6         7     3        Suspens
7         8     5             Be
8         9     2        Suspens
9        10     4        Suspens

In [48]:
# Podemos definir el intervalo que deseemos (como un Histograma)
import numpy as np
marks = np.arange(0,1.1,0.25)
print(marks)
print
print("-"*10)
factors = pd.qcut(df_random.nota, marks)
print(factors)
print("-"*10)
print(pd.value_counts(factors))
print(df_random.nota)

[0.   0.25 0.5  0.75 1.  ]
----------
0       (3.5, 5.0]
1    (-0.001, 3.0]
2    (-0.001, 3.0]
3    (-0.001, 3.0]
4       (5.0, 9.0]
5       (5.0, 9.0]
6    (-0.001, 3.0]
7       (3.5, 5.0]
8    (-0.001, 3.0]
9       (3.5, 5.0]
Name: nota, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 3.0] < (3.0, 3.5] < (3.5, 5.0] < (5.0, 9.0]]
----------
(-0.001, 3.0]    5
(3.5, 5.0]       3
(5.0, 9.0]       2
(3.0, 3.5]       0
Name: nota, dtype: int64
0    5
1    0
2    3
3    3
4    7
5    9
6    3
7    5
8    2
9    4
Name: nota, dtype: int64


## Actividad

In [16]:
# Del fichero:

import pandas as pd
df = pd.read_csv("data/rdu-weather-history.csv",sep=";")  
# print(df.head())
# print(df.columns)
print(df.shape)
print(df.snowfall.describe())

# values, repeticiones = np.unique(df.snowfall,return_counts=True)
# print(values)
# print(repeticiones)

(4557, 28)
count    4555.000000
mean        0.013723
std         0.214786
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         7.010000
Name: snowfall, dtype: float64


Queremos discretizar la variable `snowfall` para tener 4 categorias de nevada.

En primer lugar, usa la función `qcut` para hacer esta categorización. Haz una prueba y analiza por qué está función quizás no sea la mejor opción.<br/>
Ahora, crea 4 categorías utilizando la función `cut`:

In [18]:
#TODO

[![License: CC BY 4.0](https://img.shields.io/badge/License-CC_BY_4.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/) <br/>
Isaac Lera and Gabriel Moya <br/>
Universitat de les Illes Balears <br/>
isaac.lera@uib.edu, gabriel.moya@uib.edu